In [ ]:
#!pip install matplotlib numpy pmdarima sklearn statsmodels pandas plotly

In [ ]:
import pickle
import numpy as np
import statsmodels.api as sm
from matplotlib import pyplot as plt
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import kpss
import matplotlib.dates as mdates
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from plotly import graph_objs as go
from pmdarima.arima.utils import ndiffs
import logging
from pmdarima.arima import StepwiseContext
from data_util_common import train_val_test_split, show_box

In [ ]:

def check_stationary(data, lags):
    rolling_mean = data.rolling(window=lags).mean().dropna()
    rolling_std = data.rolling(window=lags).std()
    plt.plot(data, color='blue', label='Original')
    plt.plot(rolling_mean, color='red', label='Rolling Mean')
    plt.plot(rolling_std, color='black', label='Rolling Std')
    plt.legend(loc='best')
    dtFmt = mdates.DateFormatter('%Y-%b')  # define the formatting
    plt.gca().xaxis.set_major_formatter(dtFmt)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.xticks(rotation=0, fontweight='light', fontsize='x-small')
    plt.title('Rolling Mean & Rolling Standard Deviation')
    plt.show()


def kpss_test(series, **kw):
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    print(
        'KPSS checks for stochastic trend (e.g., which does not have a constant variance throughout time) stationary.')
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critical Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')
    if p_value < 0.05:
        n_kpss = ndiffs(series, test='kpss')
        print(f'N steps required for differencing: {n_kpss}')
    print()


def adf_test(series, **kw):
    result = adfuller(series, **kw)
    print('ADF checks for unit root presence.')
    print('ADF Statistic: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    print(f'num lags: {result[2]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))
    print(f'Result: The series is {"not " if result[1] > 0.05 else ""} stationary')
    if result[1] > 0.05:
        n_adf = ndiffs(series, test='adf')
        print(f'N steps required for differencing: {n_adf}')
    print()


def differencing(data, periods):
    data_diff = data.diff(periods)
    data_diff.dropna(inplace=True)
    return data_diff


def remove_seasonality(data, lag):
    seasonal_diff = data - data.shift(lag)
    seasonal_diff = seasonal_diff.dropna(inplace=False)
    return seasonal_diff


def show_result(df, title, x_title, y_title):
    fig = go.Figure()
    for col in df.columns:
        fig.add_trace(go.Bar(x=df.index, y=df[col], name=col, opacity=0.7))
    fig.update_layout(barmode='overlay')
    fig.update_layout(title=title)
    fig.update_xaxes(title=x_title)
    fig.update_yaxes(title=y_title)
    fig.write_html('sarima_result.html')
    fig.show()
    
def canopy_dataset():
    filename = 'data/NIST_Canopy_2015_2018_interpolated.csv'
    label = 'ACP_kW'
    data = pd.read_csv(filename, sep=',')
    data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S')
    data.set_index(data['TIMESTAMP'], drop=True, inplace=True)
    # data = filter_data(data, "2015-01-01", "2018-01-01")

    data[label] = data[label].astype(float)
    date = [x.strftime('%Y-%m-%d') for x in pd.to_datetime(data.index).date]
    time = [x.strftime('%H:%M:%S') for x in pd.to_datetime(data.index).time]

    print(data.columns)
    print("Correlation ----------------------------------------------------------------------------------------------")
    corr_data = data.drop(columns=[label, 'season']).corrwith(data[label])
    print(corr_data)

    pv_per_hour = data[[label]].copy()
    pv_per_hour['Time'] = time
    pv_per_hour = pv_per_hour.groupby("Time").agg({label: list}).reset_index()
    pv_per_hour = pv_per_hour.drop(columns=["Time"])
    show_box(pv_per_hour, label, "Energy distribution per hour", "Time [h]", "Energy [Wh]")
    del pv_per_hour
    return data, date, time, label

In [ ]:

pd.set_option('display.max_columns', None)
format = "%Y-%m-d"
T = 24
data, date, time, label = canopy_dataset()
features = ['Irradiance_1_Wm2']
data = data[[label] + features]

date_time = data.index
data = data.reset_index(drop=True)

# data = differencing(data, 1)
# data = remove_seasonality(data, 24)

print("**AUTOCORRELATION ** -------------------------------------------------------------------------------------")
# px.line(x=data.index, y=data[label], labels={'x': 'Time', 'y': label}).show()
# show_result(data, 'PV plant production', 'Time', label)

print("The autocorrelation fu nction (ACF) assesses the correlation between observations in a time series for a\n"
      "set of lags. In an ACF plot, each bar represents the size and direction of the correlation. Bars that\n"
      "extend across the light blue area are statistically significant. When seasonal patterns are present,\n"
      "the autocorrelations are larger for lags at multiples of the seasonal frequency than for other lags.")
plot_acf(data[label], lags=T)
plt.show()

print("The partial autocorrelation function is similar to the ACF except that it displays only the correlation\n"
      "between two observations that the shorter lags between those observations do not explain. The most \n"
      "statistically significant lags suggests the order of a autoregressive model")

plot_pacf(data[label], lags=T)
plt.show()

print("** STATIONARITY ** ---------------------------------------------------------------------------------------")
# check_stationary(data, lags)
adf_test(data[label], maxlag=T, autolag=None, regression='ct')
kpss_test(data[label], nlags=T, regression='ct')

print("** TRAINING ** -------------------------------------------------------------------------------------------")
# Create the SARIMA model
# p: The number of lag observations included in the model, also called the lag order.
# d: The number of times that the raw observations are differenced, also called the degree of differencing.
# q: The size of the moving average window, also called the order of moving average.

# The (P,D,Q,M) Order refers to the seasonal component of the model for the Auto Regressive parameters,
# differences, Moving Average parameters, and periodicity: D indicates the integration order of the seasonal
# process (the number of transformation needed to make stationary the time series). P indicates the Auto
# Regressive order for the seasonal component. Q indicated the Moving Average order for the seasonal component. M
# indicates the periodicity, i.e. the number of periods in season, such as 12 for monthly data.


split = train_val_test_split(date_time, percentage=[80, 20])
train = data[label][:split[0]]
test = data[label][split[0]:].values
exogenous_train = data[features][:split[0]]
exogenous_test = data[features][split[0]:].values
test_size = len(test)
train_size = len(train)
print(f"TRAIN SIZE {train_size} TEST SIZE {test_size}")

# EXOGENOUS DATA
sarima=None
param_search=True
if param_search:
    with StepwiseContext(max_dur=30):
        sarima = auto_arima(train.values,
                            X=exogenous_train,
                            test='adf',
                            d=None,
                            seasonal=True,
                            D=None,
                            m=T,
                            trace=True,
                            error_action='ignore',
                            suppress_warnings=True,
                            approximation=True,
                            stepwise=True,
                            maxiter=20,
                            method='cg'
                            )
    
    predictions = abs(sarima.predict(n_periods=len(test), X=exogenous_test))
else:
    # MANUAL SARIMA
    order = (2, 0, 2)  # (p, d, q)
    seasonal_order = (1, 0, 1, T)  # (P, D, Q, s)
    
    sarima = sm.tsa.SARIMAX(endog=train,
                                exog=exogenous_train,
                                order=order,
                              seasonal_order=seasonal_order
                              ).fit(low_memory=True)
    
    predictions = sarima.predict(
            exog=exogenous_test,
            start=train_size, end=train_size+test_size-1)

print("** TESTING ** --------------------------------------------------------------------------------------------")
print(sarima.summary())

print(f"LOOKBACK\n"
      f"MAE: {mean_absolute_error(test, data[label].values[train_size - T:-T])} ")

print(f"SARIMA\n"
      f"MAE: {mean_absolute_error(test, predictions)} ")

results = pd.DataFrame().from_dict({"Y_True": test, "Y_Pred": predictions})
results.index = date_time[train_size:]
results.to_csv("sarima/results.csv", sep=",")
show_result(results, "SARIMAX forecasting", "Time [h]", "Energy [Wh]", 'sarima/sarima_results.html')

logging.info("SAVING MODEL....")
with open('sarima.pkl', 'wb') as pkl:
    pickle.dump(sarima, pkl)
logging.info("Done.\n")